In [7]:
library(Seurat)
library(Matrix)
library(reticulate)
library(glmpca)
library(peakRAM)
library(scry)
library(SingleCellExperiment)
library(glmGamPoi)

# Load data

In [8]:
mtx <- readMM('/data01/hanbin973/fastrna_paper/datasets/zhengmix8eq/mat.mtx')
meta <- read.csv('/data01/hanbin973/fastrna_paper/datasets/zhengmix8eq/meta.csv', row.names=1)
colnames(mtx) <- rownames(meta)
rownames(mtx) <- 1:nrow(mtx)

# Seurat pipeline - log

In [72]:
benchmark_seurat_log <- function(obj){
    obj <- NormalizeData(obj)
    obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
    obj <- ScaleData(obj, verbose=FALSE)
    obj <- RunPCA(obj, verbose=FALSE)
    write.csv(Embeddings(obj), 'pca_coord/zhengmix8eq/pca_log.csv')
    write.csv(HVFInfo(obj)$variance.standardized, 'feature_var/zhengmix8eq/var_log.csv')
    obj
}

In [73]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_log(obj))

In [10]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,7.046,279.7


# Seurat pipeline - sct

In [80]:
benchmark_seurat_sct<- function(obj){
    var_feature <- rep(0,nrow(obj))
    names(var_feature) <- 1:nrow(obj)
    
    obj <- SCTransform(obj, variable.features.n=2000, verbose=FALSE, method='glmGamPoi')
    obj <- RunPCA(obj, verbose=FALSE)
    write.csv(Embeddings(obj), 'pca_coord/zhengmix8eq/pca_sct.csv')
    var_feature[rownames(HVFInfo(obj))] <- HVFInfo(obj, 'sct')$residual_variance
    write.csv(var_feature, 'feature_var/zhengmix8eq/var_sct.csv')
    obj
}

In [81]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_sct(obj))

In [21]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,58.516,1958.3


# glmpca

In [83]:
benchmark_scry_glmpca <- function(obj){
    obj <- devianceFeatureSelection(obj, assay="counts", sorted=FALSE)
    feature_var <- rowData(obj)$binomial_deviance
    write.csv(feature_var/mean(feature_var, na.rm=TRUE), 'feature_var/zhengmix8eq/var_scry.csv')
    select_features <- order(-feature_var)[1:2000]
    obj2 <- obj[select_features,]
    obj2 <- GLMPCA(obj2, 20, assay="counts", minibatch="stochastic")
    write.csv(metadata(obj2)$glmpca$factors, 'pca_coord/zhengmix8eq/pca_glmpca.csv')
    obj
}

In [84]:
obj <- SingleCellExperiment(list(counts=mtx))
result <- peakRAM(x <- benchmark_scry_glmpca(obj))